In [1]:
#load necessary packages
import pickle
import pandas as pd
import plotly.express as px
import matplotlib as plt

In [11]:
#Load the dataframes from the pickle object (in my private directory)
l_df_pickle = open("private/l_df.pkl", "rb")
l_df = pickle.load(l_df_pickle)
p_df_pickle = open("private/p_df.pkl", "rb")
p_df = pickle.load(p_df_pickle)
f_df_pickle = open("private/f_df.pkl", "rb")
f_df = pickle.load(f_df_pickle)

In [12]:
l_df[l_df.Name == 'German']

,ID,Name,Glottocode,Glottolog_Name,ISO639P3code,Macroarea,Latitude,Longitude,Family,dataset_ID,...,Type,Date,Transcriber,Short_Name,Region,Culture,Description,Orthography,Comment,Family_factor
396,194,German,stan1295,German,deu,Eurasia,48.649,12.4676,Indo-European,ids,...,None,None,None,None,None,None,None,None,None,Indo-European
747,deu,German,stan1295,German,deu,Eurasia,48.649,12.4676,Indo-European,northeuralex,...,None,None,None,None,None,None,None,None,None,Indo-European
1525,41700,German,stan1295,German,ger,Eurasia,48.649,12.4676,Indo-European,diacl,...,None,None,None,None,None,None,None,None,None,Indo-European


In [13]:
f_df = pd.merge(l_df[['ID', 'Glottocode']], f_df, left_on=['ID'], right_on=['Language_ID'], how='right')



In [14]:
f_df.head()

,ID_x,Glottocode,Concepticon_ID,dataset_ID,ID_y,Local_ID,Language_ID,Parameter_ID,Value,Form,...,effect,contact_situation,original_script,diacl_id,meaning,meaning_note,transliteration,ipa,Local_Orthography,Colex_Key
0,Venetianstd,vene1258,1692,logos,Venetianstd-350_friday-1,None,Venetianstd,350_friday,divendres,divendres,...,None,None,None,None,None,None,None,None,None,Venetianstd_divendres
1,Hindistd,hind1269,1692,logos,Hindistd-350_friday-1,None,Hindistd,350_friday,शुक्रवार,शुक्रवार,...,None,None,None,None,None,None,None,None,None,Hindistd_शुक्रवार
2,Romagnolstd,roma1328,1692,logos,Romagnolstd-350_friday-1,None,Romagnolstd,350_friday,vèner,vèner,...,None,None,None,None,None,None,None,None,None,Romagnolstd_vèner
3,Latinstd,lati1261,1692,logos,Latinstd-350_friday-1,None,Latinstd,350_friday,Venerisdies,Venerisdies,...,None,None,None,None,None,None,None,None,None,Latinstd_Venerisdies
4,Galicianstd,gali1258,1692,logos,Galicianstd-350_friday-1,None,Galicianstd,350_friday,venres,venres,...,None,None,None,None,None,None,None,None,None,Galicianstd_venres


In [15]:
f_df = f_df.drop(columns=['ID_x'])

,Glottocode,Concepticon_ID,dataset_ID,ID_y,Local_ID,Language_ID,Parameter_ID,Value,Form,Segments,...,effect,contact_situation,original_script,diacl_id,meaning,meaning_note,transliteration,ipa,Local_Orthography,Colex_Key
0,vene1258,1692,logos,Venetianstd-350_friday-1,None,Venetianstd,350_friday,divendres,divendres,,...,None,None,None,None,None,None,None,None,None,Venetianstd_divendres
1,hind1269,1692,logos,Hindistd-350_friday-1,None,Hindistd,350_friday,शुक्रवार,शुक्रवार,,...,None,None,None,None,None,None,None,None,None,Hindistd_शुक्रवार
2,roma1328,1692,logos,Romagnolstd-350_friday-1,None,Romagnolstd,350_friday,vèner,vèner,,...,None,None,None,None,None,None,None,None,None,Romagnolstd_vèner
3,lati1261,1692,logos,Latinstd-350_friday-1,None,Latinstd,350_friday,Venerisdies,Venerisdies,,...,None,None,None,None,None,None,None,None,None,Latinstd_Venerisdies
4,gali1258,1692,logos,Galicianstd-350_friday-1,None,Galicianstd,350_friday,venres,venres,,...,None,None,None,None,None,None,None,None,None,Galicianstd_venres


In [16]:
f_df = f_df.rename(columns={"ID_y": "ID"})
f_df.head()

,Glottocode,Concepticon_ID,dataset_ID,ID,Local_ID,Language_ID,Parameter_ID,Value,Form,Segments,...,effect,contact_situation,original_script,diacl_id,meaning,meaning_note,transliteration,ipa,Local_Orthography,Colex_Key
0,vene1258,1692,logos,Venetianstd-350_friday-1,None,Venetianstd,350_friday,divendres,divendres,,...,None,None,None,None,None,None,None,None,None,Venetianstd_divendres
1,hind1269,1692,logos,Hindistd-350_friday-1,None,Hindistd,350_friday,शुक्रवार,शुक्रवार,,...,None,None,None,None,None,None,None,None,None,Hindistd_शुक्रवार
2,roma1328,1692,logos,Romagnolstd-350_friday-1,None,Romagnolstd,350_friday,vèner,vèner,,...,None,None,None,None,None,None,None,None,None,Romagnolstd_vèner
3,lati1261,1692,logos,Latinstd-350_friday-1,None,Latinstd,350_friday,Venerisdies,Venerisdies,,...,None,None,None,None,None,None,None,None,None,Latinstd_Venerisdies
4,gali1258,1692,logos,Galicianstd-350_friday-1,None,Galicianstd,350_friday,venres,venres,,...,None,None,None,None,None,None,None,None,None,Galicianstd_venres


In [34]:
german_colex = pd.DataFrame(f_df.loc[f_df.Glottocode == 'stan1295',['Colex_Key', 'Glottocode', 'dataset_ID']])
#drop duplicates of the same form
german_colex = german_colex.drop_duplicates(subset=["Colex_Key"])
german_colex.head()

,Colex_Key,Glottocode,dataset_ID
102206,194_oft,stan1295,ids
102207,194_häufig,stan1295,ids
102552,194_immer,stan1295,ids
102553,194_stets,stan1295,ids
102917,194_bereit,stan1295,ids


In [35]:
german_colex.shape

(3117, 3)

In [36]:
#took a couple minutes
german_colex['Colex_IDs'] = german_colex.Colex_Key.map(lambda x: f_df.Concepticon_ID[f_df.Colex_Key == x].to_list())
german_colex.head()                                                       

,Colex_Key,Glottocode,dataset_ID,Colex_IDs
102206,194_oft,stan1295,ids,[1852]
102207,194_häufig,stan1295,ids,[1852]
102552,194_immer,stan1295,ids,[1676]
102553,194_stets,stan1295,ids,[1676]
102917,194_bereit,stan1295,ids,[790]


In [37]:
#choose ones that have more than one entry in colex_IDs
#many forms don't have colexifications but some have more than 2 colexified concepts!
german_colex['Num_concepts'] = german_colex.Colex_IDs.map(lambda x: len(x))
german_colex.Num_concepts.value_counts()

Num_concepts
1    2996
2     113
3       7
5       1
Name: count, dtype: int64

In [38]:
#choose only the word forms that are actually colexified
german_co = pd.DataFrame(german_colex[german_colex.Num_concepts > 1])
german_co.head()

,Colex_Key,Glottocode,dataset_ID,Colex_IDs,Num_concepts
104140,194_Ende,stan1295,ids,"[743, 742]",2
107127,194_bald,stan1295,ids,"[1963, 703]",2
109465,194_Alter,stan1295,ids,"[1789, 406]",2
127864,194_flach,stan1295,ids,"[1633, 193]",2
130847,194_groß,stan1295,ids,"[711, 1202]",2


In [51]:
german_co['Concept_names'] = german_co.Colex_IDs.map(lambda x: 
                                                     [p_df.Concepticon_Gloss[(p_df.Concepticon_ID == y)] for y in x])
                                                                                        

In [52]:
#pick first name
german_co['Concept_names'] = german_co.Concept_names.map(lambda x: 
                                                     [y.iloc[0] for y in x])
                                                                                        
german_co.head()

,Colex_Key,Glottocode,dataset_ID,Colex_IDs,Num_concepts,Concept_names
104140,194_Ende,stan1295,ids,"[743, 742]",2,"[END (OF TIME), END (OF SPACE)]"
107127,194_bald,stan1295,ids,"[1963, 703]",2,"[IMMEDIATELY, SOON]"
109465,194_Alter,stan1295,ids,"[1789, 406]",2,"[AGE, OLD MAN]"
127864,194_flach,stan1295,ids,"[1633, 193]",2,"[FLAT, SHALLOW]"
130847,194_groß,stan1295,ids,"[711, 1202]",2,"[TALL, BIG]"


In [53]:
german_co[german_co.Num_concepts > 2]

,Colex_Key,Glottocode,dataset_ID,Colex_IDs,Num_concepts,Concept_names
138497,194_Boden,stan1295,ids,"[690, 1632, 1228]",3,"[BOTTOM, FLOOR, EARTH (SOIL)]"
174916,194_tragen,stan1295,ids,"[415, 414, 413, 412, 700]",5,"[CARRY UNDER ARM, CARRY ON HEAD, CARRY ON SHOU..."
196102,194_Farbe,stan1295,ids,"[647, 157, 1613]",3,"[PAINT (PIGMENTS), DYE, COLOR]"
383280,194_wir,stan1295,ids,"[1130, 1131, 1212]",3,"[WE (EXCLUSIVE), WE (INCLUSIVE), WE]"
393083,194_Tante,stan1295,ids,"[170, 1272, 171]",3,"[FATHER'S SISTER, AUNT, MOTHER'S SISTER]"
394232,194_Onkel,stan1295,ids,"[1985, 1984, 1327]",3,"[FATHER'S BROTHER, MOTHER'S BROTHER, UNCLE]"
394233,194_Oheim,stan1295,ids,"[1985, 1984, 1327]",3,"[FATHER'S BROTHER, MOTHER'S BROTHER, UNCLE]"
1158941,41700_Haut,stan1295,diacl,"[2613, 580, 629]",3,"[SKIN (HUMAN), FUR, LEATHER]"


In [61]:
#way to combine steps
p_df.Concepticon_Gloss[p_df.Concepticon_ID == '1985'].iloc[0]

"FATHER'S BROTHER"

In [64]:
german_co['Semantic_field'] = german_co.Colex_IDs.map(lambda x: 
                                                     [p_df.Semantic_Field[(p_df.Concepticon_ID == y)].iloc[0] for y in x])
german_co.head()                                                                  

,Colex_Key,Glottocode,dataset_ID,Colex_IDs,Num_concepts,Concept_names,Semantic_field
104140,194_Ende,stan1295,ids,"[743, 742]",2,"[END (OF TIME), END (OF SPACE)]","[Time, Spatial relations]"
107127,194_bald,stan1295,ids,"[1963, 703]",2,"[IMMEDIATELY, SOON]","[Time, Time]"
109465,194_Alter,stan1295,ids,"[1789, 406]",2,"[AGE, OLD MAN]","[Time, Kinship]"
127864,194_flach,stan1295,ids,"[1633, 193]",2,"[FLAT, SHALLOW]","[Spatial relations, Spatial relations]"
130847,194_groß,stan1295,ids,"[711, 1202]",2,"[TALL, BIG]","[Spatial relations, Spatial relations]"


In [65]:
german_co.tail()

,Colex_Key,Glottocode,dataset_ID,Colex_IDs,Num_concepts,Concept_names,Semantic_field
1158941,41700_Haut,stan1295,diacl,"[2613, 580, 629]",3,"[SKIN (HUMAN), FUR, LEATHER]","[The body, The body, Clothing and grooming]"
1158965,41700_weiß,stan1295,diacl,"[1335, 1335]",2,"[WHITE, WHITE]","[Sense perception, Sense perception]"
1158969,41700_gelb,stan1295,diacl,"[1424, 1424]",2,"[YELLOW, YELLOW]","[Sense perception, Sense perception]"
1172600,41700_Jahr,stan1295,diacl,"[1276, 1318]",2,"[AUTUMN, CHICKEN]","[Time, Animals]"
1178833,41700_Schere,stan1295,diacl,"[3327, 341]",2,"[AUTUMN OR HARVEST SEASON, SICKLE]","[Time, Agriculture and vegetation]"


In [66]:
#Semantic field may not match for colexification, ontological category probably does though
german_co['Ontological_category'] = german_co.Colex_IDs.map(lambda x: 
                                                     [p_df.Ontological_Category[(p_df.Concepticon_ID == y)].iloc[0] for y in x])
german_co.head()    

,Colex_Key,Glottocode,dataset_ID,Colex_IDs,Num_concepts,Concept_names,Semantic_field,Ontological_category
104140,194_Ende,stan1295,ids,"[743, 742]",2,"[END (OF TIME), END (OF SPACE)]","[Time, Spatial relations]","[Person/Thing, Person/Thing]"
107127,194_bald,stan1295,ids,"[1963, 703]",2,"[IMMEDIATELY, SOON]","[Time, Time]","[Other, Other]"
109465,194_Alter,stan1295,ids,"[1789, 406]",2,"[AGE, OLD MAN]","[Time, Kinship]","[Person/Thing, Person/Thing]"
127864,194_flach,stan1295,ids,"[1633, 193]",2,"[FLAT, SHALLOW]","[Spatial relations, Spatial relations]","[Property, Property]"
130847,194_groß,stan1295,ids,"[711, 1202]",2,"[TALL, BIG]","[Spatial relations, Spatial relations]","[Property, Property]"


In [69]:
#only considers first two entries if more than two concepts colexified
match_sem = sum(german_co.Semantic_field.map(lambda x: x[0] == x[1]))
match_sem

71

In [70]:
#out of 121
match_ont = sum(german_co.Ontological_category.map(lambda x: x[0] == x[1]))
match_ont

109

In [84]:
#couldn't find a better way to do this
from collections import defaultdict
sem_counts = defaultdict(int)
for colex in german_co.Semantic_field.to_list(): 
    for concept in colex: 
        sem_counts[concept] += 1

In [85]:
sem_counts['Time']

11

In [86]:
sem_counts['Spatial relations']

15

In [87]:
#the values of this, arranged in a certain order and including categories not represented
#could be a feature for analysis!
sem_counts

defaultdict(int,
            {'Time': 11,
             'Spatial relations': 15,
             'Kinship': 44,
             'Emotions and values': 4,
             'The physical world': 13,
             'The house': 6,
             'Motion': 21,
             'Possession': 6,
             'Basic actions and technology': 12,
             'Cognition': 13,
             'Warfare and hunting': 2,
             'Clothing and grooming': 4,
             'Sense perception': 17,
             'The body': 24,
             'Food and drink': 17,
             'Agriculture and vegetation': 16,
             'Social and political relations': 3,
             'Speech and language': 10,
             'Animals': 10,
             'Religion and belief': 2,
             'Law': 2})

In [90]:
ont_counts = defaultdict(int)
for colex in german_co.Ontological_category.to_list(): 
    for concept in colex: 
        ont_counts[concept] += 1

In [91]:
ont_counts

defaultdict(int,
            {'Person/Thing': 139,
             'Other': 18,
             'Property': 32,
             'Action/Process': 63})